In [1]:
import pymysql
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='test_ocw',
                             charset='utf8',
                             # Selectの結果をdictionary形式で受け取る
                             cursorclass=pymysql.cursors.DictCursor)

In [2]:
import urllib.request
from bs4 import BeautifulSoup

#Q.曜日・時限が複数になった時もリスト構造にするか(今はリスト構造になっていない)
#各教員にaタグが入っている前提で以下の様になっている
#アクセスランキングは必要に感じなかったので空
def fetch_up(scrapData):
    OCW_up = {}

    for key in scrapData:
        value_list = []
        for value in key.dd.strings:
            value = value.strip().replace(" ","").replace("\n","")
            if len(value)>0: value_list.append(value)
        OCW_up[key.dt.text] = ",".join(value_list)
        
    return OCW_up

#時折text内に凄いへんな文字が入る可能性がある(元のOCWの書かれ方のせい)
#例で出たやつぶんは修正いれました
def fetch_down(scrapData):
    OCW_down = {}

    for key in scrapData:
        keyString = key.h3.text

        if key.table is not None: #授業計画・学生が身につける力
            tr_list = []
            #print([j.text for j in key.thead.tr.find_all("th")]) #ヘッダ
            for TR in key.tbody.find_all("tr"): #ボディ
                tr_list.append("[{}]".format(
                    ",".join(["\\'{}\\'".format(j.text) for j in TR.find_all("td")])
                     ))
            OCW_down[keyString] = "[{}]".format(",".join(tr_list))
            ''' リスト構造で保持する場合
            for TR in i.tbody.find_all("tr"): #ボディ
                print([j.text.split() for j in TR.find_all("td")])
            '''
            
        elif key.ul is not None: #関連する科目
            OCW_down[keyString] = ",".join([j.text for j in key.ul.find_all("li")])
            
        else: #そのほか(文章で書かれた項目)
            OCW_down[keyString] = "\n".join(key.p.strings)
        
    return OCW_down

if __name__=='__main__':
    r = urllib.request.urlopen('http://www.ocw.titech.ac.jp/index.php?module=General&action=T0300&GakubuCD=4&GakkaCD=342222&KeiCD=22&course=22&KougiCD=201804054&Nendo=2018&lang=JA&vid=03')
    h = r.read().decode('utf-8')
    soup = BeautifulSoup(h, "html.parser")
    
    OCWData = fetch_up(soup.find(attrs={"class":"gaiyo-data clearfix"}).find_all("dl"))
    OCWData.update(fetch_down(soup.find_all(attrs={"class":"cont-sec"})))
    OCWData["講義名"] = soup.find(attrs={"class":"page-title-area clearfix"}).h3.text
    print(OCWData)

{'開講元': '数理・計算科学コース', '担当教員名': '脇田建,増原英彦', '授業形態': '講義', '曜日・時限(講義室)': '月5-6(W831)\xa0\xa0木5-6(W831)', 'クラス': '-', '科目コード': 'MCS.T502', '単位数': '2', '開講年度': 'H30年度', '開講クォーター': '3Q', 'シラバス更新日': 'H30年3月20日', '講義資料更新日': '-', '使用言語': '英語', 'アクセスランキング': '', '講義の概要とねらい': '関数型プログラミングの研究・開発において培われてきたゴミ集め，型駆動の最適化，関数閉包などの要素技術は，関数型プログラミングパラダイムの壁を越え，多くのプログラミング言語に取り込まれています．関数型言語のコンパイラを読み解くことによって，高い抽象性を誇りつつも，非効率にも見える関数型のプログラムが，さまざまな技術を経て高効率な機械実行可能なコードに変換される様子を学べます．授業で扱うコンパイラは抽象構文木，K正規形，クロージャ変換，仮想機械命令などの抽象化層を経て段階的に機械命令を生成します．このように設計されたコンパクトで美しいコンパイラの構成より，さらに複雑なソフトウェアを記述するための系統的な方法についても学ぶことができるでしょう．', '到達目標': '【到達目標】\u3000本授業の目標は (1) 関数型プログラミング言語の習得，(2) 関数型プログラミングの実践的方法論の理解，(3) 関数型プログラミング言語のコンパイラの理解の三点です．関数型プログラミングができるようになること，そして関数型プログラミング言語のコンパイラのソストウェアアーキテクチャを理解することを到達目標とします．\n【テーマ】\u3000関数型プログラミング言語で記述された関数型言語のコンパイラを読み解く経験を通して，大規模なソフトウェアの構成方法，抽象化レベルの高いソフトウェアを段階的に具現化する技術，記述の抽象度と実行時の効率性を両立するための方法論など，一般の情報システム構築における要点を実践的に学びます．', 'キーワード': 'Compiler organization, Optimization, System programming, Funct

In [3]:
#データベース作ったりINSERTしたり

column = {# TOP側 #
          "科目コード":"LectureCode",
          "講義名":"LectureName",
          "担当教員名":"Professor",
          "開講元":"Course",
          "曜日・時限(講義室)":"DateRoom",
         #"講義室":"Room",
          "単位数":"TANI",
          "開講クォーター":"Quater",
          "使用言語":"Language",
          # シラバス側 #
          "授業計画・課題":"LecturePlan",
          "成績評価の基準及び方法":"HyoukaStyle",
          "履修の条件(知識・技能・履修済科目等)":"RisyuJoken"}

def createDB(column):
    with connection.cursor() as cursor:
        sub_column = []
        for k in column:
            #if column_up[k] == "講義名":
                #key処理
            #else:
            sub_column.append(column[k]+" TEXT")
        
        sql = "CREATE TABLE lecture({});".format(",".join(sub_column))
        #print(sql)
        cursor.execute(sql)
        
def dropDB():
    with connection.cursor() as cursor:
        sql = "DROP TABLE lecture;"
        cursor.execute(sql)
        
def insertLecture(column,LectureData):
    with connection.cursor() as cursor:
        sql = "INSERT INTO lecture ({}) ".format(",".join(map(lambda x:column[x],column)))
        sql += "VALUES ({});".format(",".join(map(lambda x:"\'{}\'".format(LectureData[x]),column)))
        #print(sql)
        cursor.execute(sql)

In [5]:
dropDB()
createDB(column)

In [6]:
insertLecture(column,OCWData)

In [7]:
with connection.cursor() as cursor:
    sql = "SELECT * FROM lecture"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

{'LectureCode': 'MCS.T502', 'LectureName': 'H30年度\u3000関数型プログラミング\xa0\xa0\xa0Functional Programming', 'Professor': '脇田建,増原英彦', 'Course': '数理・計算科学コース', 'DateRoom': '月5-6(W831)\xa0\xa0木5-6(W831)', 'TANI': '2', 'Quater': '3Q', 'Language': '英語', 'LecturePlan': "[['第1回','概要','Guidance'],['第2回','OCamlによる関数型プログラミングへの招待','基本的なデータ型，複合的なデータ型，代数的データ型について学ぶ'],['第3回','OCamlによる関数型プログラミングへの招待','再帰的なデータ構造，再帰的な関数，関数閉包，状態の扱い'],['第4回','OCamlによる関数型プログラミングへの招待','レコード，例外処理，モジュールシステム，標準ライブラリ，開発環境'],['第5回','関数型言語のコンパイラ MinCaml のソフトウェアアーキテクチャ','関数型言語OCamlで記述された関数型言語MinCamlのソフトウェアアーキテクチャの概要を理解する．'],['第6回','プログラムから抽象構文木へ','字句解析と構文解析について学ぶ'],['第7回','型解析','型検査と型推論，ユニフィケーション'],['第8回','抽象構文木からK正規形へ','K正規形，α変換'],['第9回','最適化(1)','β簡約，letの入れ子についての簡約，インライン展開'],['第10回','最適化(2)','定数畳み込み，不要定義除去'],['第11回','関数閉包の除去','クロージャ変換'],['第12回','仮想機械命令生成','仮想機械命令生成'],['第13回','レジスタ割り当て','レジスタ割り当て'],['第14回','実行可能コードの生成','アセンブリ生成，実行時システム'],['第15回','まとめ','まとめ']]", 'HyoukaStyle': '授業時間内の発表およびレポート課題で成績を評価する。最終試験はない。', 'RisyuJoken': '形式言語理論（

In [ ]:
#保存！
connection.commit()